### Protein Family Classification

In [27]:
import numpy as np
import pandas as pd
from collections import Counter

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.neighbors import NearestCentroid
import random

In [2]:
family_classification_metadata = pd.read_table('../seminar_5/data/family_classification_metadata.tab')
family_classification_sequences = pd.read_table('../seminar_5/data/family_classification_sequences.tab')

In [3]:
family_classification_metadata.head()

,SwissProtAccessionID,LongID,ProteinName,FamilyID,FamilyDescription
0,Q6GZX4,001R_FRG3G,Putative transcription factor 001R,Pox_VLTF3,Poxvirus Late Transcription Factor VLTF3 like
1,Q6GZX3,002L_FRG3G,Uncharacterized protein 002L,DUF230,Poxvirus proteins of unknown function
2,Q6GZX0,005R_FRG3G,Uncharacterized protein 005R,US22,US22 like
3,Q91G88,006L_IIV6,Putative KilA-N domain-containing protein 006L,DUF3627,Protein of unknown function (DUF3627)
4,Q197F3,007R_IIV3,Uncharacterized protein 007R,DUF2738,Protein of unknown function (DUF2738)


In [4]:
family_classification_sequences.head()

,Sequences
0,MAFSAEDVLKEYDRRRRMEALLLSLYYPNDRKLLDYKEWSPPRVQV...
1,MSIIGATRLQNDKSDTYSAGPCYAGGCSAFTPRGTCGKDWDLGEQT...
2,MQNPLPEVMSPEHDKRTTTPMSKEANKFIRELDKKPGDLAVVSDFV...
3,MDSLNEVCYEQIKGTFYKGLFGDFPLIVDKKTGCFNATKLCVLGGK...
4,MEAKNITIDNTTYNFFKFYNINQPLTNLKYLNSERLCFSNAVMGKI...


#### Task:
    
Use your ProtVec embedding from homework 5 to perform protein family classification using RNN.

* use 1000 most frequent families for classification
* validate your results on the train-test split
* reduce the dimensionality of the protein-space using Stochastic Neighbor Embedding and visualize two most frequent classes
* compare your RNN results with SVM
* visualization and metrics are up to you

#### Let's read the embedding matrix from the original article data.

In [5]:
table = pd.read_csv('data/protVec_100d_3grams_without_quotes.csv', sep='\t', header=None)
table = table.T
header = table.iloc[0] # grab the first row for the header
prot2vec = table[1:] # take the data less the header row
prot2vec.columns = header # set the header row as the df header
prot2vec["AAA"].head()

1    -0.17406
2   -0.095756
3    0.059515
4    0.039673
5   -0.375934
Name: AAA, dtype: object

In [6]:
most_common_families = Counter(family_classification_metadata['FamilyID']).most_common(1000)
most_common_families = [family for (family, count) in most_common_families]
family2num = {f: i for (i, f) in enumerate(most_common_families)}
family2num

{'MMR_HSR1': 0,
 'Helicase_C': 1,
 'ATP-synt_ab': 2,
 '7tm_1': 3,
 'AA_kinase': 4,
 'AAA': 5,
 'tRNA-synt_1': 6,
 'tRNA-synt_2': 7,
 'MFS_1': 8,
 'HSP70': 9,
 'Oxidored_q1': 10,
 'His_biosynth': 11,
 'Cpn60_TCP1': 12,
 'EPSP_synthase': 13,
 'Aldedh': 14,
 'Shikimate_DH': 15,
 'GHMP_kinases_N': 16,
 'Ribosomal_S2': 17,
 'Ribosomal_S4': 18,
 'Ribosomal_L16': 19,
 'KOW': 20,
 'UPF0004': 21,
 'Ribosom_S12_S23': 22,
 'GHMP_kinases_C': 23,
 'Ribosomal_S14': 24,
 'Ribosomal_S11': 25,
 'UVR': 26,
 'Ribosomal_L33': 27,
 'BRCT': 28,
 'RF-1': 29,
 'Ank_2': 30,
 'Ribosomal_L20': 31,
 'RNA_pol_Rpb2_1': 32,
 'Ribosomal_S18': 33,
 'ATP-synt_B': 34,
 'Peptidase_M20': 35,
 'Ribosomal_L18e': 36,
 'GIDA': 37,
 'Oxidored_q2': 38,
 'Ldh_1_N': 39,
 'HD': 40,
 'Ribosomal_S10': 41,
 'PALP': 42,
 'Ribosomal_L18p': 43,
 'Ribosomal_L3': 44,
 'tRNA-synt_1g': 45,
 'UbiA': 46,
 'Ribosomal_L4': 47,
 'Ribosomal_S13': 48,
 'Ribosomal_S16': 49,
 'Methyltransf_5': 50,
 'Ribosomal_L32p': 51,
 'EF_TS': 52,
 'THF_DHG_CYH':

In [7]:
MAX_PROTEIN_LEN = 501
EMBED_LEN = 100

In [8]:
all_proteins = family_classification_sequences['Sequences']
all_families = family_classification_metadata['FamilyID']

selected_ids = [i for i in range(len(all_proteins)) 
                  if all_families[i] in family2num and len(all_proteins[i]) <= MAX_PROTEIN_LEN]

random.shuffle(selected_ids)

train_ratio = 0.9
num_train = int(len(selected_ids) * train_ratio)

train_ids = selected_ids[:num_train]
test_ids = selected_ids[num_train:]

In [16]:
def embedding(protein):
    res = np.zeros(100)
    for i in range(0, (len(protein) - 3) // 3):
        try:
            res = np.add(res, prot2vec[protein[i*3: i*3 + 3]])
        except KeyError:
            res = np.add(res, prot2vec['<unk>'])

    return np.divide(res, ((len(protein) - 3) // 3))

#embedding(all_proteins[11])

In [19]:
print(len(train_ids))

177452


In [ ]:
X_train = []
for i in range(len(train_ids)):
    if i % 2000 == 0:
        print(i)
    cur_id = train_ids[i]
    X_train.append(embedding(all_proteins[cur_id]))

0
2000
4000
6000
8000
10000
12000
14000
16000
18000
20000
22000
24000
26000
28000
30000
32000
34000
36000
38000
40000
42000
44000
46000
48000
50000
52000
54000
56000
58000
60000
62000
64000
66000
68000
70000
72000
74000
76000
78000
80000


In [ ]:
print(len(test_ids))

In [ ]:
X_test = []
for i in range(len(train_ids)):
    if i % 2000 == 0:
        print(i)
    cur_id = train_ids[i]
    X_test.append(embedding(all_proteins[cur_id]))

In [39]:
y_train = all_families[train_ids]

y_test = all_families[test_ids]

for shrinkage in [None, .2, 5, 10]:
    # we create an instance of Neighbours Classifier and fit the data.
    clf = NearestCentroid(shrink_threshold=shrinkage)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(shrinkage, np.mean(y_test == y_pred))

None 0.002
0.2 0.0015
5 0.0033
10 0.0043
100 0.0003
